<h1><center>NBA Database Data Collection</center></h1>

This notebook accesses NBA Data through the [NBA API Python Package](https://pypi.org/project/nba-api/), cleans and converts the data into pandas dataframes and then stores the tables in a sqlite database. The tables will also be exported into a SQL File to be used in any SQL database. The SQL database created in this notebook will be used to create an automated dashboard in Tableau that tracks the history and current statistics of a selected team in the NBA. This project was influenced by previous work done with the [European Soccer Dataset](https://www.kaggle.com/datasets/hugomathien/soccer) from Kaggle. This dataset was created to serve as a database to be used for public use, and to be used in personal analytical projects.

In [1]:
#nba_api package
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import leaguegamelog, boxscoretraditionalv2, playercareerstats, commonteamroster, commonplayerinfo, teamdetails, boxscoresummaryv2

#analytics stack
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime
import os
import time
import urllib
import io

### Create DB and Grab Data for Tables

In [2]:
conn = sqlite3.connect('NBA.db')

In [3]:
#get player data from NBA API
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [4]:
#create players table in NBA DB
try:
    df_players.to_sql('Players', conn, index=False)
except:
    pass

In [5]:
#get team data from NBA API
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [6]:
#create team table in NBA DB
try:
    df_teams.to_sql('Team', conn, index=False)
except:
    pass

### General Game Data

In [7]:
#functions to change column names in game_data
def col_home(df):
    df.columns = [x + '_HOME' for x in df.columns]
    return df

def col_away(df):
    df.columns = [x + '_AWAY' for x in df.columns]
    return df

In [8]:
def game_data(seasons):
    #get game data from nba api
    games = leaguegamelog.LeagueGameLog(season=seasons, timeout = 100).get_data_frames()[0]
    games['TEAM_ID'] = games['TEAM_ID'].astype(str)
    
    #store season_id, game_id and game_dates in separate lists
    season_ids = games['SEASON_ID'].values[0]
    game_ids = games['GAME_ID'].unique()
    game_dates = [games['GAME_DATE'][i] for i in range(0,len(games),2)]
    
    #create new df to store revised structure
    games_s = games.drop(['GAME_ID', 'SEASON_ID', 'GAME_DATE'], axis=1)
    data = []
    df = pd.DataFrame(data)
    for i in range(0,len(games_s),2): #rename home and away columns in each row
        if "vs" in games_s['MATCHUP'].values[i]:
            home_team = col_home(games_s.iloc[[i]]).reset_index(drop = True)
            away_team = col_away(games_s.iloc[[i+1]]).reset_index(drop = True)
        else:
            home_team = col_home(games_s.iloc[[i+1]]).reset_index(drop = True)
            away_team = col_away(games_s.iloc[[i]]) .reset_index(drop = True)
        df1 = pd.concat([home_team, away_team], axis=1) # combine the renamed rows into one
        df = pd.concat([df, df1])
    cols = list(df.columns.values)
    cols = ['GAME_ID', 'SEASON_ID', 'GAME_DATE'] + cols
    
    #reassign game_id, season_id, game_date columns
    df['GAME_ID'] = game_ids
    df['SEASON_ID'] = season_ids
    df['GAME_DATE'] = game_dates
    df = df[cols]
    return df

#create df containing games for last 2 years and updates this seasons games
current_year = datetime.now().year
seasons = range(current_year-2,current_year) #new season has not started so still 2021
dfs = [game_data(season) for season in seasons]
games = pd.concat(dfs)
games.head()

,GAME_ID,SEASON_ID,GAME_DATE,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,FGM_HOME,...,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PLUS_MINUS_AWAY,VIDEO_AVAILABLE_AWAY
0,0022000001,22020,2020-12-22,1610612751,BKN,Brooklyn Nets,BKN vs. GSW,W,240,42,...,34,47,26,6,6,18,24,99,-26,1
0,0022000002,22020,2020-12-22,1610612747,LAL,Los Angeles Lakers,LAL vs. LAC,L,240,38,...,29,40,22,10,3,16,29,116,7,1
0,0022000003,22020,2020-12-23,1610612738,BOS,Boston Celtics,BOS vs. MIL,W,240,48,...,41,52,19,4,6,16,20,121,-1,1
0,0022000013,22020,2020-12-23,1610612755,PHI,Philadelphia 76ers,PHI vs. WAS,W,240,41,...,35,40,28,7,4,20,26,107,-6,1
0,0022000017,22020,2020-12-23,1610612763,MEM,Memphis Grizzlies,MEM vs. SAS,L,240,49,...,40,48,28,8,8,11,18,131,12,1


In [9]:
#create games table in NBA DB
try:
    games.to_sql('Games', conn, index=False)
except:
    pass

### Common Player Info

In [10]:
#get player ids from Player table
player_ids = df_players[df_players['is_active'] == True]['id']

def player_info(player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id, timeout = 100).get_data_frames()
    basic_info = player_info[0]
    current_stats = player_info[1]
    pi_df = pd.merge(basic_info, current_stats, how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], #merge dfs into one
                     right_on=['PLAYER_ID', 'PLAYER_NAME'])
    
    #drop redundant columns and reassign data types
    pi_df = pi_df.drop(['TimeFrame', 'PLAYER_NAME'], axis=1) 
    pi_df['PLAYER_ID'] = pi_df['PLAYER_ID'].astype(str)
    pi_df['TEAM_ID'] = pi_df['TEAM_ID'].astype(str)
    return pi_df

#create df for player info
dfs = []
for player_id in player_ids:
    time.sleep(1) #prevent connection time-out
    pi = player_info(player_id=player_id)
    dfs.append(pi)
player_info = pd.concat(dfs)
player_info.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG,PLAYER_ID,PTS,AST,REB,PIE,ALL_STAR_APPEARANCES
0,1630173,Precious,Achiuwa,Precious Achiuwa,"Achiuwa, Precious",P. Achiuwa,precious-achiuwa,1999-09-19T00:00:00,Memphis,Nigeria,...,2020,1,20,N,1630173,9.1,1.1,6.5,0.089,NaN
0,203500,Steven,Adams,Steven Adams,"Adams, Steven",S. Adams,steven-adams,1993-07-20T00:00:00,Pittsburgh,New Zealand,...,2013,1,12,N,203500,6.9,3.4,10.0,0.108,NaN
0,1628389,Bam,Adebayo,Bam Adebayo,"Adebayo, Bam",B. Adebayo,bam-adebayo,1997-07-18T00:00:00,Kentucky,USA,...,2017,1,14,N,1628389,19.1,3.4,10.1,0.161,NaN
0,1630583,Santi,Aldama,Santi Aldama,"Aldama, Santi",S. Aldama,santi-aldama,2001-01-10T00:00:00,Loyola-Maryland,Spain,...,2021,1,30,N,1630583,4.1,0.7,2.7,0.066,NaN
0,200746,LaMarcus,Aldridge,LaMarcus Aldridge,"Aldridge, LaMarcus",L. Aldridge,lamarcus-aldridge,1985-07-19T00:00:00,Texas-Austin,USA,...,2006,1,2,N,200746,19.1,1.9,8.1,NaN,7


In [11]:

try:
    players.to_sql('Player_Info', conn, index=False)
except:
    pass

### Player Regular Season & Playoff Stats

In [12]:
#get list of player_ids
player_ids = df_players[df_players['is_active'] == True]['id']

def player_stats(player_id):
    player_stats = playercareerstats.PlayerCareerStats(per_mode36 = 'PerGame', player_id=player_id).get_data_frames()
    rs = player_stats[0] #regular season stats
    ps = player_stats[2] #playoff stats
    dfs = [rs, ps]
    for df in dfs: #clean data types
        df['PLAYER_ID '] = df['PLAYER_ID'].astype(str)
        df['TEAM_ID'] = df['TEAM_ID'].astype(str)
    
    return rs, ps

#create separate dataframes for regular season and playoff stats
rs_dfs = []
ps_dfs = []
for player_id in player_ids:
    time.sleep(1) #prevent connection time-out
    rs, ps = player_stats(player_id)
    rs_dfs.append(rs)
    ps_dfs.append(ps)
    
reg_season_stats = pd.concat(rs_dfs)
playoff_stats = pd.concat(ps_dfs)

In [13]:
#create regular season and playoff stats tables
try:
    reg_season_stats.to_sql('Player_RS_Stats', conn, index=False)
except:
    pass

try:
    playoff_stats.to_sql('Player_Playoff_Stats', conn, index=False)
except:
    pass


/Users/davidshin/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


### Box Score Stats

In [14]:
def box_score_stats(game_id):
    box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
    box_score['TEAM_ID'] = box_score['TEAM_ID'].astype(str) #adjust data types
    box_score['PLAYER_ID'] = box_score['PLAYER_ID'].astype(str)
    return box_score

#create dataframe containing box score information of all games
dfs = []
for game_id in games['GAME_ID']:
    time.sleep(1) #prevent connection time-out
    b = box_score_stats(game_id=game_id)
    dfs.append(b)
box_score = pd.concat(dfs)
box_score.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022000001,1610612744,GSW,Golden State,203952,Andrew Wiggins,Andrew,F,,31:14,...,0.0,2.0,2.0,1.0,0.0,1.0,4.0,4.0,13.0,-28.0
1,0022000001,1610612744,GSW,Golden State,1629672,Eric Paschall,Eric,F,,21:33,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,6.0,-28.0
2,0022000001,1610612744,GSW,Golden State,1630164,James Wiseman,James,C,,24:17,...,1.0,5.0,6.0,0.0,2.0,0.0,1.0,2.0,19.0,-10.0
3,0022000001,1610612744,GSW,Golden State,1626162,Kelly Oubre Jr.,Kelly,G,,25:39,...,4.0,3.0,7.0,2.0,1.0,2.0,3.0,1.0,6.0,-28.0
4,0022000001,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,30:19,...,3.0,1.0,4.0,10.0,2.0,0.0,3.0,1.0,20.0,-23.0


In [15]:
#create box_score table for NBA DB
try:
    box_score.to_sql('Box_Score', conn, index=False)
except:
    pass

### Officials for Game

In [16]:
def game_officials(game_id):
    officials = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id).get_data_frames()[2]
    cols = list(officials.columns.values)
    cols = ['GAME_ID'] + cols #re-order columns
    officials['GAME_ID'] = game_id
    officials = officials[cols]
    officials['OFFICIAL_ID'] = officials['OFFICIAL_ID'].astype(str) #change data type
    return officials

#create dataframe containing officials for each game
dfs = []
for game_id in games['GAME_ID']:
    time.sleep(1) #prevent connectiontime-out 
    g = game_officials(game_id=game_id)
    dfs.append(g)
officials = pd.concat(dfs)
officials.reset_index(drop = True, inplace = True)
officials.head()

,GAME_ID,OFFICIAL_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM
0,0022000001,2534,Zach,Zarba,15
1,0022000001,202007,Nick,Buchert,3
2,0022000001,202053,Scott,Twardoski,52
3,0022000002,1193,Michael,Smith,38
4,0022000002,200833,Eric,Dalen,37


In [17]:
#create officials table for NBA DB
try:
    officials.to_sql('Officials', conn, index=False)
except:
    pass

### Database Export

In [18]:
#dump database into SQL file
conn = sqlite3.connect('NBA.db')
with io.open('backupdatabase_dump.sql', 'w') as p: 
    for line in conn.iterdump(): 
          p.write('%s\n' % line)
  
conn.close()